In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import os
from common import *
import pandas as pd
#from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import Lasso, Ridge, LinearRegression

os.chdir("../")
print(f"Root directory: {os.getcwd()}")

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "font.size": 10,
})

%matplotlib inline 

Root directory: c:\Users\andre\Dropbox\FYS-STK4155_projects\FYS-STK4155 - Project1\FYS-STK4155-Prj1_report


### Global variables 

In [2]:
INPUT_DATA = "data/input_data/"  # Path for input data
REPORT_DATA = "data/report_data/" # Path for data ment for the report
REPORT_FIGURES = "figures/" # Path for figures ment for the report

## First we plot a 3D plot of the franke function

In [3]:
# Preview plot of the franke function
%matplotlib
np.random.seed(4155)
y = x = np.arange(0, 1, 0.05)
x, y = np.meshgrid(x,y)
z = FrankeFunction(x, y)
noise_mesh = 0.05 * np.random.randn(z.shape[0], z.shape[1])
z_noisy = z + noise_mesh

fig = plt.figure()
ax = fig.add_subplot(1,2,1, projection='3d') # Are :)steike

#ax = plt.axes(projection='3d')
ax.title.set_text("Plot of the Franke Function")
ax.view_init(elev=30., azim=-25.0)
ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")
surf = ax.plot_surface(x,y,z, cmap=cm.coolwarm, linewidth = 0, antialiased=False)

ax = fig.add_subplot(1,2,2, projection='3d')
ax.title.set_text("Plot of the Franke Function (noise added)")
ax.view_init(elev=30., azim=-25.0)
ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")
surf = ax.plot_surface(x,y,z_noisy, cmap=cm.coolwarm, linewidth = 0, antialiased=False)
plt.savefig(f"{REPORT_FIGURES}franke_function_preview.pdf")

Using matplotlib backend: Qt5Agg


## Defining and creating the data

In [4]:
n = 2000 # The number of point in the franke function
x = np.random.rand(n)
y = np.random.rand(n)
z = FrankeFunction(x, y) 
noise = 0.05 * np.random.randn(n)
z += noise # Adding noise to the function values

## Approximate the franke function using ordinary least squares
We estimate the franke functinon using polynomials up to 6th degree. We than look at the MSE scores to look for overfitting. We use the MSE score values from the test data to determine overfit together with the corveture of the plots

## Plot of fit for all degrees before diciding by evaluation on which degree that yeilds the best fit

In [5]:
def randrange(n, vmin, vmax):
    return (vmax - vmin)*np.random.rand(n) + vmin

fig = plt.figure(figsize=(8,8))
degrees = 6
z_train_OLS = pd.DataFrame()
z_hat_train_OLS = pd.DataFrame()
z_test_OLS = pd.DataFrame()
z_hat_test_OLS = pd.DataFrame()

# TODO: Must fix so that training and test data are used. 
# Must evalute model using MSE from traning and test
for degree in range(1, degrees + 1):
    X = create_X(x, y, degree) # Design Matrix
    model = OLS() # The model
    model.fit(X, z) # Fitting the model
    z_hat = model.predict(X) # predict on train data
    
    # Plot
    ax = fig.add_subplot(3,2, degree, projection='3d')
    ax.view_init(elev=30., azim=-25.0)
    ax.title.set_text(f"OLS/Linear fit of degree{degree}")
    ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")
    ax.scatter3D(y, x, z_hat, c=z_hat ,marker = '.', cmap=cm.coolwarm)
fig.suptitle("OLS fit to the Franke Function")
plt.tight_layout()
plt.savefig(f"{REPORT_FIGURES}franke_function_fit_OLS.pdf")

In [6]:
def randrange(n, vmin, vmax):
    return (vmax - vmin)*np.random.rand(n) + vmin

fig = plt.figure(figsize=(8,8))
degrees = 6
z_train_OLS = pd.DataFrame()
z_hat_train_OLS = pd.DataFrame()
z_test_OLS = pd.DataFrame()
z_hat_test_OLS = pd.DataFrame()

# TODO: Must fix so that training and test data are used. 
# Must evalute model using MSE from traning and test
for degree in range(1, degrees + 1):
    X = create_X(x, y, degree) # Design Matrix
    X_train, X_test, z_train, z_test = train_test_split(X,z, test_size=0.2, shuffle=True, random_state=4155)
    
    model = OLS() # The model
    model.fit(X_train, z_train) # Fitting the model
    
    # Predictions
    z_hat_train = model.predict(X_train) # predict on train data
    z_hat_test = model.predict(X_test) # predict on test data

    # Evaluatation metrics
    # TODO:
    # Filling up dataframes
    z_train_OLS[degree] = z_train.flatten() 
    z_hat_train_OLS[degree] = z_hat_train.flatten()
    z_test_OLS[degree] = z_test.flatten()
    z_hat_test_OLS[degree] = z_hat_test.flatten()


# MSE calculations for all lambda values
mse_scores_train = ((z_train_OLS - z_hat_train_OLS) ** 2).mean()
mse_scores_test = ((z_test_OLS - z_hat_test_OLS) ** 2).mean()
# R2 calculations for all lambda values
R2_scores_train = 1 - ((z_train_OLS - z_hat_train_OLS) ** 2).sum() / ((z_train_OLS - z_train_OLS.mean())**2).sum() 
R2_scores_test = 1 - ((z_test_OLS - z_hat_test_OLS) ** 2).sum() / ((z_test_OLS - z_test_OLS.mean())**2).sum()  

# Plots
plt.figure(figsize=(12,8))
plt.plot(np.arange(1,degrees+1), mse_scores_train, label="Training data")
plt.plot(np.arange(1,degrees+1), mse_scores_test, label="Test data")
plt.xlabel("Model complexity - Number of polynomial degrees")
plt.ylabel("MSE")
plt.title("Training evaluation on OLS regression fit")
plt.legend()
plt.tight_layout()
plt.savefig(f"{REPORT_FIGURES}Franke_Function_OLS_fit_evaluation.pdf.pdf")

In [7]:
best_degree = 5
X = create_X(x, y, best_degree) # Design Matrix
model = OLS() # The model
model.fit(X, z) # Fitting the model
z_hat = model.predict(X) # predict on train data

# Plot

fig = plt.figure()#figsize=(8,8))
ax = plt.axes(projection='3d')
ax.title.set_text(f"OLS regression fit to the Franke Function\nbest degree {best_degree},")
#ax.view_init(elev=5., azim=85.0)
ax.view_init(elev=30., azim=-25.0)
ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")
ax.scatter3D(y, x, z_hat, c=z_hat ,marker = '.', cmap=cm.coolwarm)
plt.savefig(f"{REPORT_FIGURES}franke_function_best_fit_OLS.pdf")

## Approximate the franke function using Ridge regression
First we find the optimal value for the lambda parameter by splitting the input data into training and test data. Than we fit the model using different values for lambda. For each lambda value we use the mean square error on the test data evaluate how god the fit is. The best lambda value is used to fit a new model with all data (not using train-test split). 

First we find the optimal parameter value of lambda, $\lambda$, by evaluating overfit from evaluation plot where we plot the values for MSE for training and test data

In [8]:
features = 6
#x = np.arange(0, 1, 0.05)
#y = np.arange(0, 1, 0.05)

X = create_X(x, y, n=features) # Design Matrix
z = FrankeFunction(x,y)
z += noise
nbf_lambdas = 300
lambdas = np.logspace(-6,6, nbf_lambdas)
z_train_ridge = pd.DataFrame()
z_hat_train_ridge = pd.DataFrame()
z_test_ridge = pd.DataFrame()
z_hat_test_ridge = pd.DataFrame()

for lam in lambdas:
    X_train, X_test, z_train, z_test = train_test_split(X,z, test_size=0.2, shuffle=True)
    model = RidgeRegression(lam) # The model
    model.fit(X_train, z_train) # Fitting the model
   
    # Predictions
    z_hat_train = model.predict(X_train) # predict on train data
    z_hat_test = model.predict(X_test) # predict on test data

    # Filling up dataframes
    z_train_ridge[lam] = z_train.flatten() 
    z_hat_train_ridge[lam] = z_hat_train.flatten()
    z_test_ridge[lam] = z_test.flatten()
    z_hat_test_ridge[lam] = z_hat_test.flatten()


# MSE calculations for all lambda values
mse_scores_train = ((z_train_ridge - z_hat_train_ridge) ** 2).mean()
mse_scores_test = ((z_test_ridge - z_hat_test_ridge) ** 2).mean()
# R2 calculations for all lambda values
R2_scores_train = 1 - ((z_train_ridge - z_hat_train_ridge) ** 2).sum() / ((z_train_ridge - z_train_ridge.mean())**2).sum() 
R2_scores_test = 1 - ((z_test_ridge - z_hat_test_ridge) ** 2).sum() / ((z_test_ridge - z_test_ridge.mean())**2).sum()  

# Plots
plt.figure(figsize=(12,8))
plt.plot(-np.log(lambdas), mse_scores_train, label="Training data")
plt.plot(-np.log(lambdas), mse_scores_test, label="Test data")
#plt.plot(mse_scores_train,-np.log(lambdas), label="Training data")
#plt.plot(mse_scores_test,-np.log(lambdas), label="Test data")
plt.xlabel("log(lambda)")
plt.ylabel("MSE")
plt.title("Training evaluation on Ridge regression fit")
plt.legend()
plt.savefig(f"{REPORT_FIGURES}Franke_Function_Ridge_fit_evaluation.pdf")

<ipython-input-8-5c97b1ad3c48>:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  z_train_ridge[lam] = z_train.flatten()
<ipython-input-8-5c97b1ad3c48>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  z_hat_train_ridge[lam] = z_hat_train.flatten()
<ipython-input-8-5c97b1ad3c48>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  z_test_ridge[lam] = z_test.flatten()
<ipython-input-8-5c97b1ad3c48>:28: Perfo

In [9]:
lam = -1.1
model = RidgeRegression(lam) # The model
model.fit(X, z) # Fitting the model

# Predictions
z_hat = model.predict(X) # predict on train data

# Evaluatation metrics
# TODO:
results_df = pd.DataFrame(columns=["MSE", "R2-score"], index=["Training data", "Test data"])
results_df["MSE"] = MSE(z, z_hat)
results_df["R2-score"] = R2(z, z_hat)
results_df.to_csv(f"{REPORT_DATA}redge_reg_lambda_{lam}.csv")

# Plot
fig = plt.figure()#figsize=(8,8))
ax = plt.axes(projection='3d')
ax.title.set_text(f"Ridge regression fit to the Franke Function\nDegree {degree},$\lambda$:{lam}")
#ax.view_init(elev=5., azim=85.0)
ax.view_init(elev=30., azim=-25.0)
ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")
ax.scatter3D(x,y,z,c=z, cmap=cm.coolwarm, marker = '.')
plt.savefig(f"{REPORT_FIGURES}franke_function_Ridge_fit.pdf")